**Mask-RCNN setup**

In [ ]:
# uninstall improper package versions
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

In [ ]:
# reinstall with proper versions
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0

In [ ]:
# import mask rcnn and set up
%cd
!git clone https://github.com/matterport/Mask_RCNN.git
%cd Mask_RCNN/
!python setup.py install
!pip show mask-rcnn

In [ ]:
import json
import os
import sys
import numpy as np
import cv2
from mrcnn import utils
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import skimage.draw



######################################
# Configuration
######################################
class CustomConfig(Config):
    NAME = "custom_mcrnn"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 3 # ?? classes + background
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5
    LEARNING_RATE = .001
    BATCH_SIZE = 10

config = CustomConfig()


#######################################
# Dataset
#######################################
class CustomDataset(utils.Dataset):

    # define constants
    DATASET_DIR = 'data' # directory where train images can be found
    BASE_ANNOTATIONS_FILE = 'data/annotations.json' # file with annotations

    TRAIN_TEST_SPLIT = .8 # proportion of images to use for training set, remainder will be reserved for validation
    CLASSES = [] # all annotation classes, to generate from the json

    def load_dataset(self, dataset_dir, subset):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """

        #add all new classes
        for i, c in enumerate(CLASSES):
            self.add_class("object", i, c)

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(DATASET_DIR, subset)
        annotations = json.load(open(BASE_ANNOTATIONS_FILE))

        #go through each image
        for a in annotations.imgs:
            for instance in a.objects:
                if instance.polygon:
                    polygons = instance.polygon

            image_path = os.path.join(dataset_dir, a['path'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "sign",
                image_id=a["id"],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)


    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "balloon":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            all_x, all_y = zip(*p) #unzip the list
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(all_y, all_x)
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)


#######################################
# Training
#######################################

# set up train and validation data

dataset_train = CustomDataset()
dataset_train.load_dataset(validation=False)
dataset_train.prepare()

dataset_val = CustomDataset()
dataset_val.load_dataset(validation=True)
dataset_val.prepare()



urllib.request.urlretrieve("https://github.com/matterport/Mask_RCNN/releases/download/v1.0/mask_rcnn_coco.h5", "mask_rcnn_coco.h5")
model = MaskRCNN(mode='training', model_dir='./'+sys.argv[1]+'/', config=CustomConfig())

if len(sys.argv) > 2: # optionally load pre-trained weights - COCO
    model.load_weights(sys.argv[2], by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# print summary
print(model.keras_model.summary())

# train model
model.train(train_dataset=dataset_train,
            val_dataset=dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=1,
            layers='heads')

# save training results to external file
model.keras_model.save_weights(sys.argv[1]+'.h5')